In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Ttitanic Prediction Model

### Problem Definition
* Predict surviving of person
* Get insights why not survived?

# Import Libraries


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Data Understanding

In [ ]:
titanic_data = pd.read_csv("/kaggle/input/titanic/train.csv")

titanic_data.head(10)

In [ ]:
titanic_data.info()

In [ ]:
titanic_data.describe()

In [ ]:
titanic_data.isnull().sum()

# Data Cleaning and Preprocessing


### Handle Missing Data

In [ ]:
missing = titanic_data.isnull().sum()
print(missing , "\n")

total_cells = np.product(titanic_data.shape)
total_missing = missing.sum()

missing_percent = (total_missing / total_cells) * 100
print(missing_percent , "%" , sep = '')

In [ ]:
plt.hist(titanic_data["Age"] , bins = 10 , edgecolor = "black")
plt.title('Age Distribution')
plt.xlabel('Age Intervals')
plt.ylabel('Frequency')
plt.show()

#### The age is normaly distribuated so filling the null values with median is the best way

In [ ]:
median_age = titanic_data["Age"].median()

titanic_data["Age"] = titanic_data["Age"].fillna(median_age)
titanic_data.isnull().sum()

#### Drop cabin cloumn because the null values is more than 50% of data in that column 

In [ ]:
titanic_data = titanic_data.drop(columns = "Cabin" , axis = 1)
titanic_data.isnull().sum()

In [ ]:
mode_embarked = titanic_data["Embarked"].mode()
print(mode_embarked)

titanic_data["Embarked"] = titanic_data["Embarked"].fillna(mode_embarked[0])
titanic_data.isnull().sum()

### Handling Outliers


In [ ]:
sns.boxplot(x = titanic_data["Age"])

#### The age column is normaly distributed so we set boundries like below

In [ ]:
upper_boundry = titanic_data["Age"].mean() + (3 * titanic_data["Age"].std())
lower_boundry = titanic_data["Age"].mean() - (3 * titanic_data["Age"].std())
print(upper_boundry)
print(lower_boundry)

titanic_data.loc[titanic_data["Age"] >= upper_boundry , "Age"] = 68
sns.boxplot(x = titanic_data["Age"])
titanic_data["Age"].describe()

In [ ]:
plt.hist(titanic_data["Fare"] , bins = 50 ,  edgecolor = "black")
plt.title('Fare Distribution')
plt.xlabel('Fare Intervals')
plt.ylabel('Frequency')
plt.show()

#### The Fare column is skweed so we set boundries like below

In [ ]:
sns.boxplot(x = titanic_data["Fare"])

In [ ]:
iqr = (titanic_data["Fare"].quantile(0.75)) - (titanic_data["Fare"].quantile(0.25))
upper_fare_boundry = titanic_data["Fare"].quantile(0.75) + (iqr * 1.5)
lower_fare_boundry = titanic_data["Fare"].quantile(0.25) - (iqr * 1.5)
print(upper_fare_boundry)
print(lower_fare_boundry)

titanic_data["Fare"].describe()

#### Because of extrime outlier 512 the boundries is like below


In [ ]:
iqr = (titanic_data["Fare"].quantile(0.75)) - (titanic_data["Fare"].quantile(0.25))
upper_fare_boundry = titanic_data["Fare"].quantile(0.75) + (iqr * 3)
lower_fare_boundry = titanic_data["Fare"].quantile(0.25) - (iqr * 3)
print(upper_fare_boundry)
print(lower_fare_boundry)

titanic_data.loc[titanic_data["Fare"] >= upper_fare_boundry , "Fare"] = 100
sns.boxplot(x = titanic_data["Fare"])
print(titanic_data["Fare"].describe())

# Exploratory Data Analysis (EDA)

In [ ]:
titanic_data.head(10)

##### How many males and females ?

In [ ]:
gender_counts = titanic_data["Sex"].value_counts()

gender_counts.plot(kind = "bar" , color = ['blue' , 'pink'] , edgecolor = 'black')
plt.title("Gender Distribuation")
plt.xlabel("Gender")
plt.ylabel("Count")
plt.show()

##### How many survivors ?


In [ ]:
survivors_counts = titanic_data["Survived"].value_counts().sort_index()

survivors_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Survivors Distribuation")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

##### How many survivors in each male and female ?

In [ ]:
male_data = titanic_data[titanic_data["Sex"] == "male"]

male_survivors_counts = male_data["Survived"].value_counts()

male_survivors_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Survivors Male Distribuation")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

In [ ]:
female_data = titanic_data[titanic_data["Sex"] == "female"]

female_survivors_counts = female_data["Survived"].value_counts().sort_index()

female_survivors_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Survivors Female Distribuation")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

##### male survivors ?

In [ ]:
survived = male_data[male_data['Survived'] == 1]['Age']
not_survived = male_data[male_data['Survived'] == 0]['Age']

bins = range(0, 70, 5)  

plt.hist(not_survived, bins=bins, color='red', alpha=0.5, label='Not Survived')
plt.hist(survived, bins=bins, color='blue', alpha=0.5, label='Survived')


plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution of Male Passengers (Survived vs Not Survived)')
plt.legend()


plt.show()

In [ ]:
men_survivors_1 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Pclass"] == 1)]

class1_counts = men_survivors_1["Survived"].value_counts().sort_index()

class1_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Class 1 Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

men_survivors_2 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Pclass"] == 2)]

class2_counts = men_survivors_2["Survived"].value_counts().sort_index()

class2_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Class 2 Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

men_survivors_3 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Pclass"] == 3)]

class3_counts = men_survivors_3["Survived"].value_counts().sort_index()

class3_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Class 3 Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

In [ ]:
survived = men_survivors_1[men_survivors_1['Survived'] == 1]['Age']
not_survived = men_survivors_1[men_survivors_1['Survived'] == 0]['Age']

bins = range(0, 70, 5)  

plt.hist(not_survived, bins=bins, color='red', alpha=0.5, label='Not Survived')
plt.hist(survived, bins=bins, color='blue', alpha=0.5, label='Survived')


plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution of Class 1 Male Passengers (Survived vs Not Survived)')
plt.legend()


plt.show()

In [ ]:
titanic_data["Embarked"].value_counts()

In [ ]:
men_embarked1 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Embarked"] == "S")]

embarkeds_counts = men_embarked1["Survived"].value_counts().sort_index()

embarkeds_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Embarked S Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

men_embarked2 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Embarked"] == "Q")]

embarkedq_counts = men_embarked2["Survived"].value_counts().sort_index()

embarkedq_counts.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Embarked Q Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

men_embarked3 = titanic_data[(titanic_data["Sex"] == "male") & (titanic_data["Embarked"] == "C")]

embarkedc_count = men_embarked3["Survived"].value_counts().sort_index()

embarkedc_count.plot(kind = "bar" , color = ['red' , 'green'] , edgecolor = 'black')
plt.title("Embarked C Survivors of Male")
plt.xlabel("Survive")
plt.ylabel("Count")
plt.show()

In [ ]:
embarkedc_count = men_embarked3["Pclass"].value_counts().sort_index()
embarkedc_count.plot(kind = "bar" , edgecolor = 'black')
plt.title("Embarked C Survivors Clsses of Male")
plt.xlabel("Class")
plt.ylabel("Count")
plt.show()

##### Female survivors ?

In [ ]:
survived = female_data[female_data['Survived'] == 1]['Age']
not_survived = female_data[female_data['Survived'] == 0]['Age']

bins = range(0, 70, 5)  

plt.hist(not_survived, bins=bins, color='red', alpha=0.5, label='Not Survived')
plt.hist(survived, bins=bins, color='green', alpha=0.5, label='Survived')


plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution of Female Passengers (Survived vs Not Survived)')
plt.legend()


plt.show()

In [ ]:
survived = female_data[female_data['Survived'] == 1]['Parch']
not_survived = female_data[female_data['Survived'] == 0]['Parch']

bins = range(0 , 7 , 2)  

plt.hist(not_survived, bins=bins, color='red', alpha=0.5, label='Not Survived')
plt.hist(survived, bins=bins, color='green', alpha=0.5, label='Survived')


plt.xlabel('Parch')
plt.ylabel('Frequency')
plt.title('Parch Distribution of Female Passengers (Survived vs Not Survived)')
plt.legend()


plt.show()

In [ ]:
survived = female_data[female_data['Survived'] == 1]['SibSp']
not_survived = female_data[female_data['Survived'] == 0]['SibSp']

bins = range(0 , 9 , 2)  

plt.hist(not_survived, bins=bins, color='red', alpha=0.5, label='Not Survived')
plt.hist(survived, bins=bins, color='green', alpha=0.5, label='Survived')


plt.xlabel('SibSp')
plt.ylabel('Frequency')
plt.title('SibSp Distribution of Female Passengers (Survived vs Not Survived)')
plt.legend()


plt.show()

# Prepare Data For Modeling

## Train Data

#### Encoding Categorical Variables

In [ ]:
le = LabelEncoder()
titanic_data["Sex"] = le.fit_transform(titanic_data["Sex"])
titanic_data.head(10)

#### Feature Scaling


In [ ]:
scaler = StandardScaler()
titanic_data[["Age" , "Fare"]] = scaler.fit_transform(titanic_data[["Age" , "Fare"]])
titanic_data.head(10)

#### Feature Selection

In [ ]:
titanic_data = titanic_data.drop(columns = ["PassengerId" , "Name" , "Ticket" , "Embarked"])
titanic_data.head(10)
# We droped Ebarked column because titanic is sank after crossing all three distination so its not important feature

## Test Data

In [ ]:
titanic_test = pd.read_csv("/kaggle/input/titanic/test.csv")
print(titanic_test.isnull().sum())

median_test_age = titanic_test["Age"].median()

titanic_test["Age"] = titanic_test["Age"].fillna(median_test_age)

titanic_test = titanic_test.drop(columns = ["Cabin"])

mean_test_fare = titanic_test["Fare"].median()

titanic_test["Fare"] = titanic_test["Fare"].fillna(mean_test_fare)
print(titanic_test.isnull().sum())

upper_boundry2 = titanic_test["Age"].mean() + (3 * titanic_test["Age"].std())
lower_boundry2 = titanic_test["Age"].mean() - (3 * titanic_test["Age"].std())
print(upper_boundry2)
print(lower_boundry2)

titanic_test.loc[titanic_test["Age"] >= upper_boundry2 , "Age"] = 68
print(titanic_test["Age"].describe())

iqr = (titanic_test["Fare"].quantile(0.75)) - (titanic_test["Fare"].quantile(0.25))
upper_fare_boundry2 = titanic_test["Fare"].quantile(0.75) + (iqr * 3)
lower_fare_boundry2 = titanic_test["Fare"].quantile(0.25) - (iqr * 3)
print(upper_fare_boundry2)
print(lower_fare_boundry2)

titanic_test.loc[titanic_test["Fare"] >= upper_fare_boundry2 , "Fare"] = 100
print(titanic_test["Fare"].describe())

passengerid = titanic_test["PassengerId"]

titanic_test = titanic_test.drop(columns = ["PassengerId" , "Name" , "Ticket" , "Embarked"])

titanic_test["Sex"] = le.fit_transform(titanic_test["Sex"])

titanic_test[["Age" , "Fare"]] = scaler.fit_transform(titanic_test[["Age" , "Fare"]])
titanic_test.head(10)


# Train The Model

In [ ]:
y_train = titanic_data["Survived"]
x_train = titanic_data.drop(columns = ["Survived"])

model = LogisticRegression(random_state = 42 , max_iter = 1000)
model.fit(x_train , y_train)

y_train_pred = model.predict(x_train)
train_accuracy = accuracy_score(y_train , y_train_pred)
print(f'Training Accuracy: {train_accuracy}')


# Test The Model


In [ ]:
y_test_pred = model.predict(titanic_test)

df = pd.DataFrame({"PassengerId" : passengerid.values , "Survived" : y_test_pred})

df.to_csv('submission.csv', index=False)